In [78]:
from pulp import *
import pandas as pd
import numpy as np
import math

initial_loading = pd.read_csv('initial_loading.csv')
product_specs = pd.read_csv('product_specs.csv')
tam_margins = pd.read_csv('tam_and_margin.csv')
workstation_metrics = pd.read_csv('workstation_metrics.csv').T.fillna(0)

workstation_metrics.columns = workstation_metrics.iloc[0]
workstation_metrics = workstation_metrics.drop(workstation_metrics.index[0])

In [79]:
def create_abs_difference(model, x, y, name="abs_diff"):
    """
    Creates variables and constraints to represent |x - y| in a PuLP model.
    
    Parameters:
    model: PuLP LpProblem
    x: PuLP variable or number representing first value
    y: PuLP variable or number representing second value
    name: Base name for the new variables (default: "abs_diff")
    
    Returns:
    abs_diff: PuLP variable representing |x - y|
    """
    # Create a variable for the absolute difference
    abs_diff = LpVariable(f"{name}", lowBound=0)
    
    # Create positive and negative part variables
    pos_part = LpVariable(f"{name}_pos", lowBound=0)
    neg_part = LpVariable(f"{name}_neg", lowBound=0)
    
    # Add constraints
    # x - y = pos_part - neg_part
    model += x - y == pos_part - neg_part, f"{name}_decomposition"
    
    # abs_diff = pos_part + neg_part
    model += abs_diff == pos_part + neg_part, f"{name}_definition"
    
    return abs_diff

In [80]:
def tool_requirement(loading, minute_load, utilization):
    return loading * minute_load / (7 * 24 * 60 * utilization)
    
def gb_output_coef(node_gb_per_wafer, node_yield):
    return 13 * node_gb_per_wafer * node_yield / 1e9 # in billions, hence /1e9

def calculate_gb_output(loading_dict, quarter, specs_df):
    """Calculate GB output for a given quarter's loading"""
    total_gb = 0
    for node_idx, loading in loading_dict.items():
        gb_per_wafer = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_gb_per_wafer'].values[0]
        yield_rate = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_yield'].values[0]
        total_gb += loading * gb_output_coef(gb_per_wafer, yield_rate)
    return total_gb

def optimize_loading(min_node_3_wafers=0):
    """
    Optimize loading to minimize total wafers while meeting TAM with integer constraints
    
    Args:
        min_wafers: Minimum number of wafers required per node per quarter
    """
    results = {"loading": {}, "tools": {}, "capex": {}, "revenue": {}}
    quarters = product_specs['quarter'].unique()
    
    # Get initial loading
    old_loading = {
        node_idx: initial_loading[f'node{node_idx}_loading'].values[0]
        for node_idx in range(1, 4)
    }
    
    old_workstations = {
        ws: int(row['initial_tool_count']) for ws, row in workstation_metrics.iterrows()
    }
    
    results["loading"][quarters[0]] = old_loading.copy()
    results["tools"][quarters[0]] = old_workstations.copy()
    results["capex"][quarters[0]] = 0

    results["revenue"][quarters[0]] = calculate_gb_output(old_loading, quarters[0], product_specs) * 0.002 * 1e9
        
    prob = LpProblem(f"Profit_Maximization", LpMaximize)

    vars = {}
    
    # Optimize each quarter
    for i in range(1, len(quarters)):
        quarter = quarters[i]
        prev_quarter = quarters[i-1]

        vars[quarter] = {}
        
        # Decision variables - specified as integers
        node1 = LpVariable(f"node1_{quarter}", 0, None, LpInteger)
        node2 = LpVariable(f"node2_{quarter}", 0, None, LpInteger)
        node3 = LpVariable(f"node3_{quarter}", min_node_3_wafers, None, LpInteger)
        
        nodes = {
            1: node1,
            2: node2,
            3: node3
        }

        vars[quarter]["nodes"] = nodes

        workstations = {
            ws: LpVariable(f"ws{ws}_{quarter}", 0, None, LpInteger)for ws, tool_count in old_workstations.items()
        }

        for ws in workstations.keys():
            prob += workstations[ws] >= old_workstations[ws]
        
        vars[quarter]["workstations"] = workstations
        
        # Target TAM
        target_tam = tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        
        # Calculate GB output coefficients
        quarter_specs = product_specs[product_specs['quarter'] == quarter]
        coefs = {
            node_idx: gb_output_coef(
                quarter_specs[f'node{node_idx}_gb_per_wafer'].values[0],
                quarter_specs[f'node{node_idx}_yield'].values[0]
            ) 
            for node_idx in nodes.keys()
        }

        gb_output = sum([coefs[node_idx] * node for node_idx, node in nodes.items()])
        revenue = gb_output * tam_margins[tam_margins['quarter'] == quarter]['contribution_margin'].values[0] * 1e9

        # 3. Workstation tool count
        for ws, ws_metrics in workstation_metrics.iterrows():
            tool_requirements = [
                tool_requirement(
                    node, 
                    ws_metrics[f"node_{node_idx}_minute_load"], 
                    ws_metrics[f"utilization"],
                ) for node_idx, node in nodes.items()
            ]

            prob += workstations[ws] >= sum(tool_requirements), f"WS_min_{ws}_{quarter}"

        capex = sum((workstations[ws] - old_workstations[ws]) * workstation_metrics['capex_per_tool'][ws] for ws in workstations)

        prob += revenue - capex, f"Maximize_Profit_{quarter}"

        # Constraints
        # 1. Quarter to quarter loading change constraint (±2500)
        for node_idx, node in nodes.items():
            prob += node >= old_loading[node_idx] - 2500, f"Node{node_idx}_min_change_{quarter}"
            prob += node <= old_loading[node_idx] + 2500, f"Node{node_idx}_max_change_{quarter}"
        
        # 2. TAM constraints (±2B from target)
        prob += gb_output >= target_tam - 2, f"Min_TAM_{quarter}"
        prob += gb_output <= target_tam + 2, f"Max_TAM_{quarter}"
        
        # create_abs_difference(prob, target_tam, gb_output, f"abs_tams_diff_{quarter}")

        old_workstations = workstations
        old_loading = nodes

    print(vars)
    
    # Solve
    status = prob.solve()
    
    if status != 1:
        print(f"\nWarning: Could not find optimal solution")
        print(f"Status code: {status}")
        print("Solver messages:")
        print(prob.solver.problem.status())

    for i in range(1, len(quarters)):
        quarter = quarters[i]
        prev_quarter = quarters[i-1]
        q_vars = vars[quarter]
        nodes = q_vars["nodes"]
        workstations = q_vars["workstations"]
        
        # Extract loading values (rounding to nearest integer)
        loading_values = { node_idx: int(round(nodes[node_idx].varValue)) 
                           for node_idx in nodes }
        results["loading"][quarter] = loading_values
        
        # Extract workstation tool counts
        tool_count_values = { ws: int(round(workstations[ws].varValue))
                              for ws in workstations }
        results["tools"][quarter] = tool_count_values
        
        # Calculate capex as the incremental cost from previous quarter's tool counts
        prev_tool_counts = results["tools"][prev_quarter]
        capex_value = sum((tool_count_values[ws] - prev_tool_counts[ws]) *
                          workstation_metrics['capex_per_tool'][ws]
                          for ws in tool_count_values)
        results["capex"][quarter] = capex_value
    
        # Recompute revenue:
        # First, get the GB output coefficients for this quarter
        quarter_specs = product_specs[product_specs['quarter'] == quarter]
        coefs = {
            node_idx: gb_output_coef(
                quarter_specs[f'node{node_idx}_gb_per_wafer'].values[0],
                quarter_specs[f'node{node_idx}_yield'].values[0]
            )
            for node_idx in loading_values
        }
        # Calculate total GB output (in billions)
        gb_output_value = sum(loading_values[node_idx] * coefs[node_idx]
                              for node_idx in loading_values)
        # Get the contribution margin and compute revenue
        contribution_margin = tam_margins[tam_margins['quarter'] == quarter] \
                              ['contribution_margin'].values[0]
        revenue_value = gb_output_value * contribution_margin * 1e9
        results["revenue"][quarter] = revenue_value
    
    return results, vars

def validate_solution(results):
    """Validate that solution meets all constraints with detailed output"""
    quarters = list(results['loading'].keys())
    violations = []
    
    for i in range(len(quarters)):
        quarter = quarters[i]
        
        # Check integer and non-negative loading
        for node in range(1, 4):
            if not isinstance(results['loading'][quarter][node], int):
                violations.append(f"{quarter}: {node} has non-integer loading: {results['loading'][quarter][node]}")
            if results['loading'][quarter][node] < 0:
                violations.append(f"{quarter}: {node} has negative loading: {results['loading'][quarter][node]}")
        
        # Check quarter-to-quarter change
        if i > 0:
            prev_quarter = quarters[i-1]
            for node in range(1, 4):
                change = results['loading'][quarter][node] - results['loading'][prev_quarter][node]
                if abs(change) > 2500:
                    violations.append(
                        f"{quarter}: {node} change exceeds limit: {change:+d} "
                        f"(from {results['loading'][prev_quarter][node]} to {results['loading'][quarter][node]})"
                    )
        
        # Check TAM constraints
        gb_output = calculate_gb_output(results['loading'][quarter], quarter, product_specs)
        target_tam = tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        if abs(gb_output - target_tam) > 2:
            violations.append(
                f"{quarter}: TAM deviation exceeds 2B: output={gb_output:.2f}B, "
                f"target={target_tam:.2f}B, diff={gb_output-target_tam:+.2f}B"
            )
    
    if violations:
        print("\nConstraint Violations:")
        for v in violations:
            print(f"- {v}")
        return False
    
    print("\nAll constraints satisfied:")
    print(f"- Integer loading values ✓")
    print(f"- Non-negative loading ✓")
    print(f"- Quarter-to-quarter changes within ±2500 ✓")
    print(f"- TAM requirements met within ±2B ✓")
    return True

In [81]:
def tool_requirement(loading, minute_load, utilization):
    return loading * minute_load / (7 * 24 * 60 * utilization)
    
def gb_output_coef(node_gb_per_wafer, node_yield):
    return 13 * node_gb_per_wafer * node_yield / 1e9  # in billions, hence /1e9

def calculate_gb_output(loading_dict, quarter, specs_df, gb_coefficients=None):
    """
    Calculate GB output for a given quarter's loading
    
    Args:
        loading_dict: Dictionary of node loadings
        quarter: Quarter to calculate for
        specs_df: DataFrame containing specifications
        gb_coefficients: Pre-calculated coefficients (optional optimization)
    """
    if gb_coefficients and quarter in gb_coefficients:
        return sum(loading * gb_coefficients[quarter][node_idx] 
                  for node_idx, loading in loading_dict.items())
    
    total_gb = 0
    for node_idx, loading in loading_dict.items():
        gb_per_wafer = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_gb_per_wafer'].values[0]
        yield_rate = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_yield'].values[0]
        total_gb += loading * gb_output_coef(gb_per_wafer, yield_rate)
    return total_gb

def optimize_loading(min_node_3_wafers=0):
    """
    Optimize loading to minimize total wafers while meeting TAM with integer constraints
    
    Args:
        min_node_3_wafers: Minimum number of wafers required for node 3 per quarter
    """
    results = {"loading": {}, "tools": {}, "capex": {}, "revenue": {}}
    quarters = product_specs['quarter'].unique()
    
    # Pre-calculate all coefficients
    gb_coefficients = {
        quarter: {
            node_idx: gb_output_coef(
                product_specs[product_specs['quarter'] == quarter][f'node{node_idx}_gb_per_wafer'].values[0],
                product_specs[product_specs['quarter'] == quarter][f'node{node_idx}_yield'].values[0]
            )
            for node_idx in range(1, 4)
        }
        for quarter in quarters
    }
    
    # Pre-calculate TAM targets and contribution margins
    tam_targets = {
        quarter: tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        for quarter in quarters
    }
    
    contribution_margins = {
        quarter: tam_margins[tam_margins['quarter'] == quarter]['contribution_margin'].values[0]
        for quarter in quarters
    }
    
    # Store initial values
    old_loading = {
        node_idx: initial_loading[f'node{node_idx}_loading'].values[0]
        for node_idx in range(1, 4)
    }
    
    old_workstations = {
        ws: int(row['initial_tool_count']) 
        for ws, row in workstation_metrics.iterrows()
    }
    
    # Set initial quarter results
    results["loading"][quarters[0]] = old_loading.copy()
    results["tools"][quarters[0]] = old_workstations.copy()
    results["capex"][quarters[0]] = 0
    results["revenue"][quarters[0]] = calculate_gb_output(
        old_loading, quarters[0], product_specs, gb_coefficients
    ) * contribution_margins[quarters[0]] * 1e9
    
    # Create optimization problem
    prob = LpProblem("Profit_Maximization", LpMaximize)
    
    # Create all variables upfront
    vars = {
        quarter: {
            "nodes": {
                1: LpVariable(f"node1_{quarter}", 0, None, LpInteger),
                2: LpVariable(f"node2_{quarter}", 0, None, LpInteger),
                3: LpVariable(f"node3_{quarter}", min_node_3_wafers, None, LpInteger)
            },
            "workstations": {
                ws: LpVariable(f"ws{ws}_{quarter}", 0, None, LpInteger)
                for ws in workstation_metrics.index
            }
        }
        for quarter in quarters[1:]  # Skip first quarter as it uses initial values
    }
    
    # Add all constraints and objective components
    objective = 0
    for i in range(1, len(quarters)):
        quarter = quarters[i]
        prev_quarter = quarters[i-1]
        q_vars = vars[quarter]
        nodes = q_vars["nodes"]
        workstations = q_vars["workstations"]
        
        for ws, ws_metrics in workstation_metrics.iterrows():
            # Pre-calculate minute loads and utilization
            loads = [ws_metrics[f"node_{node_idx}_minute_load"] for node_idx in range(1, 4)]
            util = ws_metrics["utilization"]
            
            tool_req = sum(
                tool_requirement(nodes[node_idx], loads[node_idx-1], util)
                for node_idx in range(1, 4)
            )
            
            prob += workstations[ws] >= tool_req, f"WS_min_{ws}_{quarter}"
            
            # Compare against previous quarter's workstation count instead of initial
            prev_ws_count = (results["tools"][prev_quarter][ws] if i == 1 
                            else vars[prev_quarter]["workstations"][ws])
            prob += workstations[ws] >= prev_ws_count, f"WS_prev_{ws}_{quarter}"
        
        # Quarter-to-quarter loading changes
        for node_idx in range(1, 4):
            prev_loading = (results["loading"][prev_quarter][node_idx] 
                          if i == 1 else vars[prev_quarter]["nodes"][node_idx])
            prob += nodes[node_idx] >= prev_loading - 2500, f"Node{node_idx}_min_change_{quarter}"
            prob += nodes[node_idx] <= prev_loading + 2500, f"Node{node_idx}_max_change_{quarter}"
        
        # TAM constraints using pre-calculated coefficients
        gb_output = sum(nodes[node_idx] * gb_coefficients[quarter][node_idx] 
                       for node_idx in range(1, 4))
        
        prob += gb_output >= tam_targets[quarter] - 2, f"Min_TAM_{quarter}"
        prob += gb_output <= tam_targets[quarter] + 2, f"Max_TAM_{quarter}"
        
        # Calculate revenue and capex for objective
        revenue = gb_output * contribution_margins[quarter] * 1e9
        capex = sum((workstations[ws] - (results["tools"][prev_quarter][ws] if i == 1 
                    else vars[prev_quarter]["workstations"][ws])) 
                    * workstation_metrics['capex_per_tool'][ws] 
                    for ws in workstations)
        
        objective += revenue - capex
    
    # Set objective
    prob += objective, "Total_Profit"
    
    # Solve
    solver = PULP_CBC_CMD(msg=False, timeLimit=600)  # Set time limit to 10 minutes
    status = prob.solve(solver)
    
    if status != 1:
        print(f"\nWarning: Could not find optimal solution")
        print(f"Status code: {status}")
        return None, None
    
    # Extract results
    for i in range(1, len(quarters)):
        quarter = quarters[i]
        prev_quarter = quarters[i-1]
        q_vars = vars[quarter]
        
        # Extract loading values
        results["loading"][quarter] = {
            node_idx: int(round(q_vars["nodes"][node_idx].varValue))
            for node_idx in range(1, 4)
        }
        
        # Extract tool counts
        results["tools"][quarter] = {
            ws: int(round(q_vars["workstations"][ws].varValue))
            for ws in q_vars["workstations"]
        }
        
        # Calculate capex
        results["capex"][quarter] = sum(
            (results["tools"][quarter][ws] - results["tools"][prev_quarter][ws])
            * workstation_metrics['capex_per_tool'][ws]
            for ws in results["tools"][quarter]
        )
        
        # Calculate revenue using pre-calculated coefficients
        gb_output = calculate_gb_output(
            results["loading"][quarter], 
            quarter, 
            product_specs,
            gb_coefficients
        )
        results["revenue"][quarter] = gb_output * contribution_margins[quarter] * 1e9
    
    return results, vars

def validate_solution(results):
    """Validate that solution meets all constraints with detailed output"""
    quarters = list(results['loading'].keys())
    violations = []
    
    # Pre-calculate TAM targets for validation
    tam_targets = {
        quarter: tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        for quarter in quarters
    }
    
    for i in range(len(quarters)):
        quarter = quarters[i]
        
        # Check integer and non-negative loading
        for node in range(1, 4):
            loading = results['loading'][quarter][node]
            if not isinstance(loading, int):
                violations.append(f"{quarter}: Node {node} has non-integer loading: {loading}")
            if loading < 0:
                violations.append(f"{quarter}: Node {node} has negative loading: {loading}")
        
        # Check quarter-to-quarter change
        if i > 0:
            prev_quarter = quarters[i-1]
            for node in range(1, 4):
                change = results['loading'][quarter][node] - results['loading'][prev_quarter][node]
                if abs(change) > 2500:
                    violations.append(
                        f"{quarter}: Node {node} change exceeds limit: {change:+d} "
                        f"(from {results['loading'][prev_quarter][node]} to {results['loading'][quarter][node]})"
                    )
        
        # Check TAM constraints
        gb_output = calculate_gb_output(results['loading'][quarter], quarter, product_specs)
        target_tam = tam_targets[quarter]
        if abs(gb_output - target_tam) > 2:
            violations.append(
                f"{quarter}: TAM deviation exceeds 2B: output={gb_output:.2f}B, "
                f"target={target_tam:.2f}B, diff={gb_output-target_tam:+.2f}B"
            )
    
    if violations:
        print("\nConstraint Violations:")
        for v in violations:
            print(f"- {v}")
        return False
    
    print("\nAll constraints satisfied:")
    print(f"- Integer loading values ✓")
    print(f"- Non-negative loading ✓")
    print(f"- Quarter-to-quarter changes within ±2500 ✓")
    print(f"- TAM requirements met within ±2B ✓")
    return True

In [82]:
results, vars = optimize_loading(min_node_3_wafers = 0)

# Validate solution
if validate_solution(results):
    print("\nAll constraints satisfied")
else:
    print("\nSome constraints violated")

# Convert results to DataFrame for easy viewing
# qn1_part_b_results_df = pd.DataFrame(results).T
# qn1_part_b_results_df.index.name = 'quarter'

# qn1_part_b_results_df


Constraint Violations:
- Q1'26: Node 1 has non-integer loading: 12000
- Q1'26: Node 2 has non-integer loading: 5000
- Q1'26: Node 3 has non-integer loading: 1000

Some constraints violated


In [83]:
results

{'loading': {"Q1'26": {1: 12000, 2: 5000, 3: 1000},
  "Q2'26": {1: 12006, 2: 6900, 3: 3500},
  "Q3'26": {1: 9640, 2: 8969, 3: 6000},
  "Q4'26": {1: 8178, 2: 9486, 3: 7096},
  "Q1'27": {1: 6981, 2: 10080, 3: 7994},
  "Q2'27": {1: 6981, 2: 10080, 3: 7994},
  "Q3'27": {1: 6878, 2: 9917, 3: 8033},
  "Q4'27": {1: 6860, 2: 10078, 3: 7995}},
 'tools': {"Q1'26": {'A': 10,
   'B': 18,
   'C': 5,
   'D': 11,
   'E': 15,
   'F': 2,
   'G': 23,
   'H': 3,
   'I': 4,
   'J': 1},
  "Q2'26": {'A': 13,
   'B': 33,
   'C': 10,
   'D': 12,
   'E': 18,
   'F': 8,
   'G': 30,
   'H': 3,
   'I': 8,
   'J': 3},
  "Q3'26": {'A': 13,
   'B': 33,
   'C': 10,
   'D': 12,
   'E': 18,
   'F': 8,
   'G': 30,
   'H': 3,
   'I': 8,
   'J': 3},
  "Q4'26": {'A': 13,
   'B': 33,
   'C': 10,
   'D': 12,
   'E': 18,
   'F': 8,
   'G': 30,
   'H': 3,
   'I': 8,
   'J': 3},
  "Q1'27": {'A': 13,
   'B': 33,
   'C': 10,
   'D': 12,
   'E': 18,
   'F': 8,
   'G': 30,
   'H': 3,
   'I': 8,
   'J': 3},
  "Q2'27": {'A': 13,
   '

In [84]:
results['capex']

{"Q1'26": 0,
 "Q2'26": 202200000.0,
 "Q3'26": 0.0,
 "Q4'26": 0.0,
 "Q1'27": 0.0,
 "Q2'27": 0.0,
 "Q3'27": 0.0,
 "Q4'27": 0.0}

In [86]:
pd.DataFrame(results["tools"]).T

,A,B,C,D,E,F,G,H,I,J
Q1'26,10,18,5,11,15,2,23,3,4,1
Q2'26,13,33,10,12,18,8,30,3,8,3
Q3'26,13,33,10,12,18,8,30,3,8,3
Q4'26,13,33,10,12,18,8,30,3,8,3
Q1'27,13,33,10,12,18,8,30,3,8,3
Q2'27,13,33,10,12,18,8,30,3,8,3
Q3'27,13,33,10,12,18,8,30,3,8,3
Q4'27,13,33,10,12,18,8,30,3,8,3


In [94]:
pd.DataFrame(results["loading"]).to_csv('part_b_ans.csv')
pd.DataFrame(results["loading"]).T

,1,2,3
Q1'26,12000,5000,1000
Q2'26,12006,6900,3500
Q3'26,9640,8969,6000
Q4'26,8178,9486,7096
Q1'27,6981,10080,7994
Q2'27,6981,10080,7994
Q3'27,6878,9917,8033
Q4'27,6860,10078,7995


In [88]:
pd.DataFrame(calcs["tools"]).T

,A,B,C,D,E,F,G,H,I,J
Q1'26,10,18,5,11,15,2,23,3,4,1
Q2'26,12,24,8,12,16,4,33,4,4,7
Q3'26,13,29,9,12,16,6,38,4,4,7
Q4'26,14,32,10,12,16,7,43,4,4,8
Q1'27,14,33,11,12,16,7,44,4,4,8
Q2'27,14,33,11,12,16,7,44,4,4,8
Q3'27,14,33,11,12,16,7,45,4,4,8
Q4'27,14,33,11,12,16,7,45,4,4,8


In [89]:
results["capex"]

{"Q1'26": 0,
 "Q2'26": 202200000.0,
 "Q3'26": 0.0,
 "Q4'26": 0.0,
 "Q1'27": 0.0,
 "Q2'27": 0.0,
 "Q3'27": 0.0,
 "Q4'27": 0.0}

In [90]:
results["revenue"]

{"Q1'26": 43680000.00000001,
 "Q2'26": 58799988.0,
 "Q3'26": 72534865.0,
 "Q4'26": 81999995.99999999,
 "Q1'27": 92789970.0,
 "Q2'27": 104013546.0,
 "Q3'27": 108999995.00000001,
 "Q4'27": 110999798.0}

In [91]:
profits = [results['revenue'][quarter] - results['capex'][quarter] for quarter in results['capex'].keys()]
net_profits = sum(profits)
print(f"{net_profits:,}")

471,618,158.0


In [93]:
# profits = [calcs['revenue'][quarter] - calcs['capex'][quarter] for quarter in calcs['capex'].keys()]
# net_profits = sum(profits)
# print(f"{net_profits:,}")